In [42]:
Nmax =0; Nmin = 0;
S = 1/2; I1 = 3/2; I2 = 1/2; 

BN = 6480.0; DN = 5.544e-3;
gamma = 81.03; 
bBa = 2301; cBa = 75; bFBa = bBa + cBa/3;
#Aparallel = 2376.0; Aperp = 2301.0; bFBa = 1/3 (Aparallel + 2 Aperp); cBa = Aparallel - Aperp;
bF = 60.0; cF = 0; bFF = bF + cF/3;
eq0Q = -117;
gS = 2.002; gI2 = 5.258; gI1 = 0.94/1.5; grot = 0.048; glp = -0.006 ;
uB = 1.39961 ;
uN = uB/1836;

In [43]:
import numpy as np
from sympy.physics.wigner import wigner_6j,wigner_9j,wigner_3j,clebsch_gordan
from numpy import linalg as LA

def kdel(x,y):
    if x==y:
        return 1
    else:
        return 0
def reduced(x):
    return np.sqrt(x*(x+1)*(2*x+1))
def nreduced(x,y):
    return np.sqrt((2*x+1)*(2*y+1))

def HN(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return kdel(mF,mFp)*kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*kdel(G,Gp)*(BN*N*(N+1)-DN*N**2*(N+1)**2)

def HNS(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return (gamma*kdel(mF,mFp)*kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*reduced(N)*reduced(S)*nreduced(G,Gp)*(-1)**(F1+I1+N+S+1+Gp+Gp)
            *wigner_6j(Np,Gp,F1,G,N,1)*wigner_6j(S,Gp,I1,G,S,1))

def HFBa(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return bFBa/2*kdel(mF,mFp)*kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*kdel(G,Gp)*(G*(G+1)-S*(S+1)-I1*(I1+1))

def HCBa(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    val=0.0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+F1+I1+N+Jp)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)*
                    wigner_6j(I1,Jp,F1,J,I1,1)*wigner_9j(J,Jp,1,N,Np,2,S,S,1))
    return (-cBa*np.sqrt(30)/3*kdel(mF,mFp)*kdel(F,Fp)*kdel(F1,F1p)**reduced(I1)*reduced(S)*nreduced(G,Gp)*
            nreduced(N,Np)*wigner_3j(N,2,Np,0,0,0)*val)

def HQ(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    val=0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+F1+I1+S+Jp)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)*
                    wigner_6j(I1,Jp,F1,J,I1,2)*wigner_6j(Np,Jp,S,J,N,2))
    return (eq0Q/4*kdel(mF,mFp)*kdel(F,Fp)*kdel(F1,F1p)*nreduced(G,Gp)*nreduced(N,Np)*
            wigner_3j(N,2,Np,0,0,0)*1/(wigner_3j(I1,2,I1,-I1,0,I1))*val)

def HFF(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return (bFF*kdel(mF,mFp)*kdel(F,Fp)*kdel(N,Np)*(-1)**(F+I2+N+G+Gp+I1+S)*nreduced(G,Gp)*nreduced(F1,F1p)*reduced(I2)*
            reduced(S)*wigner_6j(I2,F1p,F,F1,I2,1)*wigner_6j(Gp,F1p,N,F1,G,1)*wigner_6j(S,Gp,I1,G,S,1))

def HCF(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    val=0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+F+I2+I1+1+N+J)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)
                    *wigner_6j(I2,F1p,F,F1,I2,1)*wigner_6j(Jp,F1p,I1,F1,J,1)*wigner_9j(J,Jp,1,N,Np,2,S,S,1))
    return (-cF*np.sqrt(30)/3*kdel(mF,mFp)*kdel(F,Fp)*reduced(I2)*reduced(S)*nreduced(G,Gp)*
            nreduced(N,Np)*nreduced(F1,F1p)*wigner_3j(N,2,Np,0,0,0)*val)
def HZS(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return (gS*uB*(-1)**(F-mF)*wigner_3j(F,1,Fp,-mF,0,mFp)*(-1)**(Fp+I2+F1+1)*nreduced(F,Fp)*nreduced(F1,F1p)*nreduced(G,Gp)*
            (-1)**(F1p+N+G+1)*kdel(N,Np)*wigner_6j(F1p,Fp,I2,F,F1,1)*wigner_6j(Gp,F1p,N,F1,G,1)*(-1)**(Gp+I1+S+1)*
            wigner_6j(S,Gp,I1,G,S,1)*reduced(S))
def HZI1(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return (-gI1*uN*(-1)**(F-mF)*wigner_3j(F,1,Fp,-mF,0,mFp)*(-1)**(Fp+I2+F1+1)*nreduced(F,Fp)*nreduced(F1,F1p)*nreduced(G,Gp)*
            (-1)**(F1p+N+G+1)*kdel(N,Np)*wigner_6j(F1p,Fp,I2,F,F1,1)*wigner_6j(Gp,F1p,N,F1,G,1)*(-1)**(G+I1+S+1)*
            wigner_6j(I1,Gp,S,G,I1,1)*reduced(I1))
def HZI2(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return (-gI2*uN*(-1)**(F-mF)*wigner_3j(F,1,Fp,-mF,0,mFp)*(-1)**(F+I2+F1+1)*nreduced(F,Fp)*kdel(F1,F1p)
            *wigner_6j(I2,Fp,F1,F,I2,1)*reduced(I2))
def HZN(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    return (-grot*uB*(-1)**(F-mF)*wigner_3j(F,1,Fp,-mF,0,mFp)*(-1)**(Fp+I2+F1+1)*nreduced(F,Fp)*nreduced(F1,F1p)
            *wigner_6j(F1p,Fp,I2,F,F1,1)*kdel(G,Gp)*
            (-1)**(F1+Np+G+1)*wigner_6j(Np,F1p,G,F1,N,1)*reduced(N)*kdel(N,Np))

def HZgl(state):
    (G,N,F1,F,mF,Gp,Np,F1p,Fp,mFp)=state
    val=0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            for sigma in np.arange(-S,S+1):
                for sigmap in np.arange(-S,S+1):
                    for q in [-1,1]:
                        val += ((-1)**(G+Gp)*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)*nreduced(J,Jp)**2
                                 *nreduced(G,Gp)*(-1)**(N-S+Np-S+sigma+sigmap)*nreduced(N,Np)*wigner_3j(J,S,N,sigma,-sigma,0)
                                 *wigner_3j(Jp,S,Np,sigmap,-sigmap,0)*(-1)**(F-mF)*wigner_3j(F,1,Fp,-mF,0,mFp)*
                                 (-1)**(Fp+I2+F1+1)*nreduced(F,Fp)*wigner_6j(F1p,Fp,I2,F,F1,1)*
                                 (-1)**(F1p+J+I1+1)*nreduced(F1,F1p)*wigner_6j(Jp,F1p,I1,F1,J,1)*
                                 (-1)**(J+S-2*sigma)*wigner_3j(J,1,Jp,-sigma,1,sigmap)*wigner_3j(S,1,S,-sigma,1,sigmap)
                                 *reduced(S))
    return val*glp*uB
                        


def H_noB(state):
    return HN(state)+HNS(state)+HFBa(state)+HCBa(state)+HFF(state)+HCF(state)
def H_justB(state):
    return HZS(state)+HZI1(state)+HZI2(state)+0*HZN(state)+HZgl(state)

In [44]:
#Construct coupled basis
myStates=[[0,0,0,0,0]];
for n in np.arange(Nmin,Nmax+1):
    for g in np.arange(np.abs(I1-S),I1+S+1):
        for f1 in np.arange(np.abs(n-g),n+g+1):
            for f in np.arange(np.abs(f1-I2),f1+I2+1):
                for mf in np.arange(-f,f+1):
                    myStates=np.append(myStates,[[g,n,f1,f,mf]],axis=0)
myStates=np.delete(myStates,0,axis=0)

In [45]:
#Construct the Hamiltonian
#Hailtonian without field
H_0=np.zeros([len(myStates),len(myStates)])
H_B=np.zeros([len(myStates),len(myStates)])
for m in range(len(myStates)):
    for n in range(len(myStates)):
        a=np.append(myStates[m,:],myStates[n,:])
        H_0[m,n]=H_noB(a)
        H_B[m,n]=H_justB(a)

In [46]:
fil=115
B_list=np.arange(fil,fil+1,1)
toWrite=np.zeros([1+len(myStates),len(B_list)])
col_count=0
for i in B_list:
    H=H_0+i*H_B
    w, v = LA.eig(H)
    w=np.round(w,decimals=5)
    v=np.round(v,decimals=1)
    sorted_w=sorted(w)
    c=np.append(i,sorted_w)
    toWrite[:,col_count]=c
    col_count += 1
np.savetxt('test.csv',toWrite, delimiter=',', fmt='%10f') 

In [47]:
def matSort(M,index):
    m=len(index)
    C=np.zeros([m,m])
    for i in range(m):
        C[:,i]=M[:,index[i]]
    return C
sort_index = np.argsort(w)
sorted_w=sorted(w)
sorted_v=matSort(v,sort_index)

In [48]:
#View energy levels with the state composition
for m in range(len(w)):
    n=-m-1
    rr=sorted_v[:,n]
    res = [idx for idx, val in enumerate(rr) if val != 0]
    print(sorted_w[n],end = " ; ")
    for i in res:
        if (i != res[-1]):
            print(rr[i],str(myStates[i,:]),end =" + " )
        else:
            print(rr[i],str(myStates[i,:]))

1919.98131 ; 1.0 [2.  0.  2.  2.5 2.5]
1894.09902 ; -0.8 [2.  0.  2.  1.5 1.5] + -0.6 [2.  0.  2.  2.5 1.5]
1833.40919 ; 0.6 [2.  0.  2.  1.5 1.5] + -0.8 [2.  0.  2.  2.5 1.5]
1825.30771 ; 0.6 [2.  0.  2.  1.5 0.5] + 0.8 [2.  0.  2.  2.5 0.5]
1753.09142 ; -0.5 [ 2.   0.   2.   1.5 -0.5] + -0.9 [ 2.   0.   2.   2.5 -0.5]
1746.60451 ; 0.8 [2.  0.  2.  1.5 0.5] + -0.6 [2.  0.  2.  2.5 0.5]
1677.67896 ; -0.3 [ 2.   0.   2.   1.5 -1.5] + -0.9 [ 2.   0.   2.   2.5 -1.5]
1657.88846 ; -0.9 [ 2.   0.   2.   1.5 -0.5] + 0.5 [ 2.   0.   2.   2.5 -0.5]
1599.01869 ; 1.0 [ 2.   0.   2.   2.5 -2.5]
1566.43787 ; -0.9 [ 2.   0.   2.   1.5 -1.5] + 0.3 [ 2.   0.   2.   2.5 -1.5]
-2823.62967 ; 0.9 [ 1.   0.   1.   0.5 -0.5] + -0.5 [ 1.   0.   1.   1.5 -0.5]
-2837.98827 ; -1.0 [ 1.   0.   1.   1.5 -1.5]
-2912.88714 ; 0.7 [1.  0.  1.  0.5 0.5] + -0.7 [1.  0.  1.  1.5 0.5]
-2913.24033 ; 0.5 [ 1.   0.   1.   0.5 -0.5] + 0.9 [ 1.   0.   1.   1.5 -0.5]
-2985.13496 ; 0.7 [1.  0.  1.  0.5 0.5] + 0.7 [1.  0.  1.  

In [49]:
H

array([[-2838.93297863,     0.        ,     0.        ,   -37.71463401,
            0.        ,     0.        ,     0.        ,   146.80942148,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ],
       [    0.        , -2946.06702137,     0.        ,     0.        ,
          -37.71463401,     0.        ,     0.        ,     0.        ,
          146.80942148,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        , -2834.30375627,     0.        ,
            0.        ,     0.        ,    33.23858738,     0.        ,
            0.        ,     0.        ,     0.        ,   124.57192496,
            0.        ,     0.        ,     0.        ,     0.        ],
       [  -37.71463401,     0.        ,     0.        , -2888.10125209,
            0.        ,     0.        ,     0.        ,    -8